In [8]:
import json
import os
import numpy as np
import pickle
import pandas as pd
from django.shortcuts import render, redirect
from django.http import HttpResponse
from django.template import loader

#from .forms import CareForm


prediction_columns = column_afters
column_current = column_current
almost_prediction_columns = list(prediction_columns[1]) + list(prediction_columns[2]) + list(prediction_columns[3])
path_to_models_dir = "./create_model/trained_models/"

In [12]:
input_dict = pd.DataFrame.from_dict(data_dict, orient='index').T
input_dict = input_dict[input_columns]
input_dict = input_dict.astype('int64')


In [13]:
input_dict

,sex,age,disease,days,helper,meal,hygienic,wipingClean,upperBodyDressing,lowerBodyDressing,...,bedsChairsWheelchairs,toilet,bathtubShower,walkingWheelchair,stairs,understanding,expression,socialCommunication,problemSolving,memory
0,1,70,2,4,1,4,3,4,4,4,...,4,3,3,3,2,5,5,6,5,5


In [14]:
results={}
results_home={}
#学習は0-6でしているため合わせる
input_dict[column_current] = input_dict[column_current] - 1

for length_value in range(len(prediction_columns)): #4

    for col in prediction_columns[length_value]:
        name =  col + "_LGBM.pkl"
        filename = os.path.join(path_to_models_dir, name)

        with open(filename, 'rb') as web:
            loaded_model = pickle.load(web)

         #テストデータの予測
        if length_value == 0:
            predicted_home = loaded_model.predict(input_dict)

            y_pred_home = []
            for x in predicted_home:
                y_pred_home.append(np.round(x))

            y_pred_home = np.array(y_pred_home)
            results_home["predict_home"] = [predicted_home[0], y_pred_home[0]]


        else:
            predicted = loaded_model.predict(input_dict)
            y_pred= np.argmax(predicted, axis=1)  # 最尤と判断したクラスの値にする
            results[col] = y_pred[0]

output_df = pd.DataFrame.from_dict(results, orient='index').T
output_df[almost_prediction_columns] = output_df[almost_prediction_columns] + 1


df_sum_score, df_score = _acurate_sum(output_df, input_dict)
df_graph = pd.DataFrame(np.array(df_sum_score),  index=["現在", "１ヶ月後予測", "2ヶ月後予測", "3ヶ月後予測"],columns=["FIM合計点数"])

present = int(df_sum_score[0])
after_1M = int(df_sum_score[1])
after_2M = int(df_sum_score[2])
after_3M = int(df_sum_score[3])



/Users/tomoyasu.sano/anaconda3/envs/djangoai/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [15]:
print(present, after_1M, after_2M, after_3M)

55 90 105 90


In [16]:
df_graph

,FIM合計点数
現在,55
１ヶ月後予測,90
2ヶ月後予測,105
3ヶ月後予測,90


In [17]:
df_score.index

Index(['meal', 'hygienic', 'wipingClean', 'upperBodyDressing',
       'lowerBodyDressing', 'toiletAction', 'urinationControl',
       'defecationControl', 'bedsChairsWheelchairs', 'toilet', 'bathtubShower',
       'walkingWheelchair', 'stairs', 'understanding', 'expression',
       'socialCommunication', 'problemSolving', 'memory'],
      dtype='object')

In [18]:
df_graph.values

array([[ 55],
       [ 90],
       [105],
       [ 90]])

## グラフ表示

In [19]:
import plotly.express as px
import plotly.figure_factory as ff

fig = px.line(df_graph,x=df_graph.index, y="FIM合計点数",  title='FIM予測', hover_name=df.index, )
fig.show()

NameError: name 'df' is not defined

In [23]:
help(px)

Help on package plotly.express in plotly:

NAME
    plotly.express

DESCRIPTION
    `plotly.express` is a terse, consistent, high-level wrapper around `plotly.graph_objects`
    for rapid data exploration and figure generation. Learn more at https://plotly.express/

PACKAGE CONTENTS
    _chart_types
    _core
    _doc
    _imshow
    _special_inputs
    colors (package)
    data (package)

CLASSES
    builtins.object
        plotly.express._special_inputs.Constant
        plotly.express._special_inputs.IdentityMap
        plotly.express._special_inputs.Range
    
    class Constant(builtins.object)
     |  Constant(value, label=None)
     |  
     |  Objects of this class can be passed to Plotly Express functions that expect column
     |  identifiers or list-like objects to indicate that this attribute should take on a
     |  constant value. An optional label can be provided.
     |  
     |  Methods defined here:
     |  
     |  __init__(self, value, label=None)
     |      Initial

In [24]:
fig_table = ff.create_table(df_score, height_constant=60,index=True,index_title='FIM 項目' )
fig_table.layout.width=250
#fig_table.show()

In [174]:
?px.line

In [11]:
def _acurate_sum(output_df, input_dict):
    score_0M = input_dict[column_current]
    score_1M = output_df[sum_1M]
    score_2M = output_df[sum_2M]
    score_3M = output_df[sum_3M]

    #column名を揃える
    score_1M.columns = column_current
    score_2M.columns = column_current
    score_3M.columns = column_current


    #print(type(hoge_dict))
    sum_0M_score = score_0M.sum(axis=1)
    sum_1M_score = score_1M.sum(axis=1)
    sum_2M_score = score_2M.sum(axis=1)
    sum_3M_score = score_3M.sum(axis=1)

    df_sum_score = [sum_0M_score, sum_1M_score,sum_2M_score,sum_3M_score]

    df_score = pd.concat([score_0M, score_1M, score_2M, score_3M]).T
    df_score.columns = ["現在", "１ヶ月後予測", "2ヶ月後予測", "3ヶ月後予測"]
    
    return df_sum_score, df_score


In [10]:
data_dict = {
    "sex":1,
    "age":70,
    "disease":2,
    "days":4,
    "helper":1,
    "meal":4,
    "hygienic":3,
    "wipingClean":4,
    "upperBodyDressing":4,
    "lowerBodyDressing":4,
    "toiletAction":3,
    "urinationControl":5,
    "defecationControl":5,
    "bedsChairsWheelchairs":4,
    "toilet":3,
    "bathtubShower":3,
    "walkingWheelchair":3,
    "stairs":2,
    "understanding":5,
    "expression":5,
    "socialCommunication":6,
    "problemSolving":5,
    "memory":5
}

In [3]:
input_columns = [
    'sex','age','disease','days','helper','meal','hygienic','wipingClean','upperBodyDressing','lowerBodyDressing',
    'toiletAction','urinationControl','defecationControl','bedsChairsWheelchairs','toilet','bathtubShower','walkingWheelchair',
    'stairs','understanding','expression','socialCommunication','problemSolving','memory',
]

attributes = ['sex','age','disease','days','helper']

column_current =['meal','hygienic','wipingClean','upperBodyDressing','lowerBodyDressing','toiletAction','urinationControl','defecationControl',
            'bedsChairsWheelchairs','toilet','bathtubShower','walkingWheelchair','stairs','understanding','expression',
            'socialCommunication','problemSolving','memory']

column_current_ja = ['食事','整容','清拭','更衣上半身','更衣下半身','トイレ動作','排尿管理','排便管理','ベッド・椅子・車椅子移乗','トイレ移乗','浴槽・シャワー移乗','歩行・車椅子','階段','理解','表出','社会的交流','問題解決','記憶']

column_afters =[
    ['discharge'],
    ['meal_after_1M','hygienic_after_1M','wipingClean_after_1M','upperBodyDressing_after_1M','lowerBodyDressing_after_1M','toiletAction_after_1M','urinationControl_after_1M',
    'defecationControl_after_1M','bedsChairsWheelchairs_after_1M','toilet_after_1M','bathtubShower_after_1M','walkingWheelchair_after_1M',
    'stairs_after_1M','understanding_after_1M','expression_after_1M','socialCommunication_after_1M','problemSolving_after_1M','memory_after_1M'],
    ['meal_after_2M','hygienic_after_2M','wipingClean_after_2M','upperBodyDressing_after_2M','lowerBodyDressing_after_2M',
    'toiletAction_after_2M','urinationControl_after_2M','defecationControl_after_2M','bedsChairsWheelchairs_after_2M','toilet_after_2M',
    'bathtubShower_after_2M','walkingWheelchair_after_2M','stairs_after_2M','understanding_after_2M','expression_after_2M','socialCommunication_after_2M',
    'problemSolving_after_2M','memory_after_2M'],
    ['meal_after_3M','hygienic_after_3M','wipingClean_after_3M','upperBodyDressing_after_3M','lowerBodyDressing_after_3M',
    'toiletAction_after_3M','urinationControl_after_3M','defecationControl_after_3M','bedsChairsWheelchairs_after_3M','toilet_after_3M',
    'bathtubShower_after_3M','walkingWheelchair_after_3M','stairs_after_3M','understanding_after_3M','expression_after_3M','socialCommunication_after_3M',
    'problemSolving_after_3M','memory_after_3M']
    ]


# predict colums 
sum_1M = ['meal_after_1M','hygienic_after_1M','wipingClean_after_1M','upperBodyDressing_after_1M','lowerBodyDressing_after_1M','toiletAction_after_1M','urinationControl_after_1M',
    'defecationControl_after_1M','bedsChairsWheelchairs_after_1M','toilet_after_1M','bathtubShower_after_1M','walkingWheelchair_after_1M',
    'stairs_after_1M','understanding_after_1M','expression_after_1M','socialCommunication_after_1M','problemSolving_after_1M','memory_after_1M'
]
sum_2M = ['meal_after_2M','hygienic_after_2M','wipingClean_after_2M','upperBodyDressing_after_2M','lowerBodyDressing_after_2M',
    'toiletAction_after_2M','urinationControl_after_2M','defecationControl_after_2M','bedsChairsWheelchairs_after_2M','toilet_after_2M',
    'bathtubShower_after_2M','walkingWheelchair_after_2M','stairs_after_2M','understanding_after_2M','expression_after_2M','socialCommunication_after_2M',
    'problemSolving_after_2M','memory_after_2M'
    ]

sum_3M =['meal_after_3M','hygienic_after_3M','wipingClean_after_3M','upperBodyDressing_after_3M','lowerBodyDressing_after_3M',
    'toiletAction_after_3M','urinationControl_after_3M','defecationControl_after_3M','bedsChairsWheelchairs_after_3M','toilet_after_3M',
    'bathtubShower_after_3M','walkingWheelchair_after_3M','stairs_after_3M','understanding_after_3M','expression_after_3M','socialCommunication_after_3M',
    'problemSolving_after_3M','memory_after_3M'
]

In [30]:
staff = ["加藤さんが、2020年3月に担当したK様と状態がとてもよく似ています。困った時は相談をしてみてはいかがでしょうか",
         "田中さんが、2019年10月に担当したH様と状態が少し似ています。何かヒントをもらってみましょう！",
         "小松さんが、2018年2月に担当したA様と状態がとてもよく似ています。困った時は相談をしてみてはいかがでしょうか",
           "岩澤さんが、2017年5月に担当したP様と状態が少し似ています。何かヒントをもらってみましょう！",
           "後藤さんが、2020年1月に担当したW様と状態がとてもよく似ています。困った時は相談をしてみてはいかがでしょうか",
           "南川さんが、2018年7月に担当したM様と状態がとてもよく似ています。何かヒントをもらってみましょう！",
          "横山さんが、2019年12月に担当したC様と状態がとてもよく似ています。困った時は相談をしてみてはいかがでしょうか",    
        ]

In [32]:
import random
random.choice(staff)

'後藤さんが、2020年1月に担当したW様と状態がとてもよく似ています。困った時は相談をしてみてはいかがでしょうか'